# Building a quantum lock using phase kickback

## Review of phase kickback

Phase kickback is a powerful quantum phenomenon that uses entanglement properties to allow for the transfer of phase information from a target register to a control qubit. It plays a vital role in the design of many quantum algorithms.

In a phase kickback circuit, an ancilla qubit is prepared in a superposition state using a Hadamard gate and it acts as a control qubit for a controlled unitary gate applied to the target register. When the target register is in an eigenstate of the unitary gate, the corresponding eigenvalue’s phase is “kicked back” to the control qubit. A subsequent Hadamard gate on the ancilla qubit enables the extraction of the phase information through measurement.

<p style="text-align: center"><img src="images/PhaseKickback.png" width="200" height="100"></p>

## Building the quantum lock

 The quantum lock is represented by a unitary $U$, which has all but one eigenvalue equal to 1. Our one “key” eigenstate has eigenvalue -1: 

<p style="text-align: center">$U\ket{key} = -\ket{key}$</p>

When the correct eigenstate is input, the -1 phase imparted by $U$ is kicked back to the ancilla, effectively changing its state from $\ket{+}$ to $\ket{-}$. Then the outcome of the measurement on the control qubit tells us whether the correct eigenstate was inputted or not. In this case, $\ket{1} = H\ket{-}$ represents unlocking the lock, and $\ket{0} = H\ket{+}$ represents failure. To make things simple, here we’ll work with a lock-in computational basis. In this setting, the key corresponds to a binary encoded integer $m$, which will be our key eigenstate:

<p style="text-align: center">
    $\begin{split}U|n\rangle =
      \begin{cases}
      -|n\rangle, & \text{if } n=m \\
      |n\rangle, & \text{if } n\neq m
      \end{cases}\end{split}$
<\p>

## Opening the quantum lock

To open the quantum lock, the correct input state or “quantum key” is needed. Let’s see how the quantum system evolves when the right key is inputted.

A Hadamard gate is first applied to the control qubit:

<p style="text-align: center">
    $\frac{|0\rangle|\text{key}\rangle + |1\rangle|\text{key}\rangle}{\sqrt{2}}$
</p>

Then, the controlled unitary gate is applied:

<p style="text-align: center">
    $\frac{|0\rangle|\text{key}\rangle - |1\rangle|\text{key}\rangle}{\sqrt{2}} = |-\rangle|\text{key}\rangle$
</p>

Finally, a Hadamard gate is applied again to the control qubit:

<p style="text-align: center">
    $|1\rangle|\text{key}\rangle$
</p>

## Instructions to create the quantum lock

1. Import the necessary libraries and create a device to run our quantum circuits.
2. Use 5 qubits with qubit [0] as the control ancilla qubit, and qubits [1,2,3,4] as the target qubits where $\ket{\psi}$ will be encoded.
3. Create a function using [Flipsign](https://docs.pennylane.ai/en/stable/code/api/pennylane.FlipSign.html) to build the lock.
4. Prepare the corresponding eigenstate for a key we want to try out. Remember, the lock is only unlocked by the “key” eigenstate with eigenvalue -1. Make use of [BasisState](https://docs.pennylane.ai/en/stable/code/api/pennylane.BasisState.html) to build the key.
5. Create the circuit with Hadamard and control (qml.ctrl) gates.
6. Create an output function with different messages if the key is locked (state 1) or unlocked.
7. Finally, test your code with different input arrays.

In [9]:
import pennylane as qml
import numpy as np

num_wires = 5
dev = qml.device("default.qubit", wires=num_wires, shots=1)

def quantum_lock(secret_key):
    return qml.FlipSign(secret_key, wires=list(range(1, num_wires)))

def build_key(key):
    return qml.BasisState(key, wires=list(range(1, num_wires)))

@qml.qnode(dev)
def quantum_locking_mechanism(lock, key):
    build_key(key)
    qml.Hadamard(wires=0)  # Hadamard on ancilla qubit
    qml.ctrl(lock, control=0)  # Controlled unitary operation
    qml.Hadamard(wires=0)  # Hadamard again on ancilla qubit
    return qml.sample(wires=0)


def check_key(lock, key):
    if quantum_locking_mechanism(lock, key) == 1:
        print("Great job, you uncovered the right key!")
    else:
        print("Nice try, but that's not the right key!")
        
secret_key = np.array([0, 1, 1, 1])
lock = quantum_lock(secret_key)
check_key(lock, secret_key)

incorrect_key = np.array([1, 1, 1, 1])
check_key(lock, incorrect_key)

Great job, you uncovered the right key!
Nice try, but that's not the right key!
